In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import random
import scipy.stats as stats
from scipy.stats import binom
import scipy
import itertools
from collections import Counter
from collections import defaultdict
import functools
import Bio
from Bio import motifs
import multiprocessing as mp
from sklearn import metrics
from sklearn.cross_validation import train_test_split
from sklearn import ensemble
import math
import bokeh
from bokeh.charts import Chord
from bokeh.io import show, output_file
%matplotlib inline

C:\Users\Wolf\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
#low expression populations
s1path ="C:\\Users\\Wolf\\Desktop\\SynPro\\Complete raw data for sorted cells\\S1.fa.uniq.tcga.all.csv"
s3path ="C:\\Users\\Wolf\\Desktop\\SynPro\\Complete raw data for sorted cells\\S3.fa.uniq.tcga.all.csv"
s5path ="C:\\Users\\Wolf\\Desktop\\SynPro\\Complete raw data for sorted cells\\S5.fa.uniq.tcga.all.csv"
#high expression populations
s2path ="C:\\Users\\Wolf\\Desktop\\SynPro\\Complete raw data for sorted cells\\S2.fa.uniq.tcga.all.csv"
s4path ="C:\\Users\\Wolf\\Desktop\\SynPro\\Complete raw data for sorted cells\\S4.fa.uniq.tcga.all.csv"
s6path ="C:\\Users\\Wolf\\Desktop\\SynPro\\Complete raw data for sorted cells\\S6.fa.uniq.tcga.all.csv"
s7path ="C:\\Users\\Wolf\\Desktop\\SynPro\\Complete raw data for sorted cells\\S7.fa.uniq.tcga.all.csv"

In [2]:
#low expression populations
s1path ="Z:/PROJECTS/Synthetic Promoters/Library#1/Miseq data/Complete raw Data for sorted cells/S1.fa.uniq.tcga.all.csv"
s3path ="Z:/PROJECTS/Synthetic Promoters/Library#1/Miseq data/Complete raw Data for sorted cells/S3.fa.uniq.tcga.all.csv"
s5path ="Z:/PROJECTS/Synthetic Promoters/Library#1/Miseq data/Complete raw Data for sorted cells/S5.fa.uniq.tcga.all.csv"
#high expression populations
s2path ="Z:/PROJECTS/Synthetic Promoters/Library#1/Miseq data/Complete raw Data for sorted cells/S2.fa.uniq.tcga.all.csv"
s4path ="Z:/PROJECTS/Synthetic Promoters/Library#1/Miseq data/Complete raw Data for sorted cells/S4.fa.uniq.tcga.all.csv"
s6path ="Z:/PROJECTS/Synthetic Promoters/Library#1/Miseq data/Complete raw Data for sorted cells/S6.fa.uniq.tcga.all.csv"
#restimulated population
s7path ="Z:/PROJECTS/Synthetic Promoters/Library#1/Miseq data/Complete raw Data for sorted cells/S7.fa.uniq.tcga.all.csv"

In [3]:
#Retreive previously calculated tres for dataset
sm_csv = 'c:/users/wolf/desktop/p7_tredf.csv'
#sm_csv = 'Z:/PROJECTS/Synthetic Promoters/Library#1/Hits Testing/p7_tredf.csv'
sm_df1 = pd.read_csv(sm_csv)
#del sm_df1['Unnamed: 0']
sm_df1.head()

,seq_num,tre,position,logprob
0,0,10,51,10.755157
1,0,69,-105,11.726842
2,0,71,37,13.255016
3,0,71,64,13.255016
4,0,71,-43,13.255016


In [4]:
#libaddress = 'Z:/PROJECTS/Synthetic Promoters/Library#1/Hits Testing/Jaspar_lib.txt'
libaddress = 'C:/Users/Wolf/Documents/BioScripts/Jaspar_lib.txt'
fh = open(libaddress)

#Collect all TRE names, in order
with open(libaddress, 'r') as f:
    name_lst = []
    for ln in f:
        if ln.startswith('>'):
            name_lst.append(ln[10:-1])

#get frequency matrix for each TRE
pssms = [m.counts.normalize(pseudocounts=0.25).log_odds() for m in motifs.parse(fh, "jaspar")]

In [5]:
#Calc length of each TRE, put into df
tre_lens = [len(pwm[0]) for pwm in pssms]
tre_len_dict = list(zip(name_lst, tre_lens))
trelen_df = pd.DataFrame(tre_len_dict).reset_index()
trelen_df.columns = ['tre', 'tre_name', 'tre_len']
trelen_df.head()

,tre,tre_name,tre_len
0,0,RUNX1,11
1,1,TFAP2A,11
2,2,Arnt,6
3,3,Ahr::Arnt,6
4,4,Ar,17


In [6]:
#read in a csv with a single column, split to divide '# of hits' from nucleotide sequence, filter out TREs with < 100 hits
def read_in(filepath):
    dfs1 = pd.read_csv(filepath, engine='python', sep=None, header=None)
    dfs1.columns = ['count','sequence']
    dfs1 = dfs1.loc[dfs1['count'] > 20]
    return dfs1

raw_df = read_in(s7path)
raw_df['sequence'].replace(regex=True,inplace=True,to_replace=r'TCGAGTAGAGTCTAGACTCTACATTTTGACACCCCCA',value=r'')
raw_df = raw_df.reset_index()
raw_df['seq_len'] = raw_df['sequence'].str.len()
raw_df.head()

,index,count,sequence,seq_len
0,0,184563,TCGAATGACATCATCTTTCGAAGATCAAAGGGTCGAGGGGACTTTC...,195
1,1,80600,TCGAGGGGACTTTCCTCGATGTGACTCATTCGATTTCTTGGAAATC...,269
2,2,78625,TCGATTTCCAAGAAATCGATTTCTTGGAAATCGAATGACATCATCT...,235
3,3,70196,TCGATCTCCGCCCCCTCTTCGAAGATCAAAGGGTCGAGGGGACTTT...,183
4,4,66217,TCGAGGGGACTTTCCTCGAGGAAAGTCCCCTCGATTTCTTGGAAAT...,220


In [7]:
#merge raw df for sequence and count with tre analysis data
treseqdf = pd.merge(sm_df1, raw_df, left_on='seq_num', right_on='index', how='inner')
treseqdf.head()

,seq_num,tre,position,logprob,index,count,sequence,seq_len
0,0,10,51,10.755157,0,184563,TCGAATGACATCATCTTTCGAAGATCAAAGGGTCGAGGGGACTTTC...,195
1,0,69,-105,11.726842,0,184563,TCGAATGACATCATCTTTCGAAGATCAAAGGGTCGAGGGGACTTTC...,195
2,0,71,37,13.255016,0,184563,TCGAATGACATCATCTTTCGAAGATCAAAGGGTCGAGGGGACTTTC...,195
3,0,71,64,13.255016,0,184563,TCGAATGACATCATCTTTCGAAGATCAAAGGGTCGAGGGGACTTTC...,195
4,0,71,-43,13.255016,0,184563,TCGAATGACATCATCTTTCGAAGATCAAAGGGTCGAGGGGACTTTC...,195


In [8]:
#merge with tre df to get true names and lengths of tres, calculate corrected position
jsel_df = pd.merge(treseqdf, trelen_df, on='tre', how='inner')
jsel_df['sequence'] = jsel_df['sequence'].apply(lambda x: x.upper())
jsel_df['cposition'] = np.where(jsel_df['position'] >= 0, jsel_df['position'], 
                                ( jsel_df['sequence'].str.len() + jsel_df['position']))

In [9]:
jsel_df.head()

,seq_num,tre,position,logprob,index,count,sequence,seq_len,tre_name,tre_len,cposition
0,0,10,51,10.755157,0,184563,TCGAATGACATCATCTTTCGAAGATCAAAGGGTCGAGGGGACTTTC...,195,E2F1,12,51
1,3,10,84,10.755157,3,70196,TCGATCTCCGCCCCCTCTTCGAAGATCAAAGGGTCGAGGGGACTTT...,183,E2F1,12,84
2,4,10,49,10.755157,4,66217,TCGAGGGGACTTTCCTCGAGGAAAGTCCCCTCGATTTCTTGGAAAT...,220,E2F1,12,49
3,4,10,-89,10.755157,4,66217,TCGAGGGGACTTTCCTCGAGGAAAGTCCCCTCGATTTCTTGGAAAT...,220,E2F1,12,131
4,6,10,-116,10.755157,6,37540,TCGAATGAGTCACATCGAGCGCCAAATCGAGGGGACTTTCCTCGAG...,130,E2F1,12,14


In [10]:
sm_nadd = pd.merge(sm_df1, trelen_df, on='tre', how='inner')
del sm_nadd['tre']
del sm_nadd['tre_len']
sm_nadd.head()

,seq_num,position,logprob,tre_name
0,0,51,10.755157,E2F1
1,3,84,10.755157,E2F1
2,4,49,10.755157,E2F1
3,4,-89,10.755157,E2F1
4,6,-116,10.755157,E2F1


In [11]:
sm_nadd.loc[sm_nadd['tre_name'] == 'CREB1']

,seq_num,position,logprob,tre_name
99414,345,107,12.892933,CREB1
99415,345,-88,12.892933,CREB1
99416,504,5,12.892933,CREB1
99417,504,-190,12.892933,CREB1
99418,657,124,12.892933,CREB1
99419,657,-71,12.892933,CREB1
99420,729,183,12.892933,CREB1
99421,729,-12,12.892933,CREB1


In [12]:
grp_df = sm_nadd[['seq_num', 'tre_name']].groupby(['seq_num', 'tre_name']).size().reset_index()
grp_df.columns = ['seq_num', 'tre_name', 'count']
pivdf = grp_df.pivot(index='seq_num', columns= 'tre_name').reset_index()
pivdf.columns = pivdf.columns.droplevel(0)
pivdf.rename(columns={'':'seq_num'}, inplace=True)
pivdf.fillna(value=0, inplace=True)
pivdf.head()

tre_name,seq_num,ARNT::HIF1A,ATF4,ATF7,Arid3a,Atf1,Atf3,BATF3,BATF::JUN,Bcl6,...,TFAP2C,TFAP2C(var.2),TFAP2C(var.3),Tcf7,VENTX,YY1,ZBTB7B,ZBTB7C,ZNF263,ZNF740
0,0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,6.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0
3,3,0.0,0.0,4.0,0.0,0.0,0.0,0.0,1.0,2.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0,5.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
pivdf['CREB1'].sum()

8.0

In [14]:
anadf = pd.merge(raw_df, pivdf, left_on='index', right_on = 'seq_num')
del anadf['sequence']
del anadf['seq_num']
del anadf['index']
anadf.head()

,count,seq_len,ARNT::HIF1A,ATF4,ATF7,Arid3a,Atf1,Atf3,BATF3,BATF::JUN,...,TFAP2C,TFAP2C(var.2),TFAP2C(var.3),Tcf7,VENTX,YY1,ZBTB7B,ZBTB7C,ZNF263,ZNF740
0,184563,195,0.0,0.0,8.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,80600,269,0.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
2,78625,235,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0
3,70196,183,0.0,0.0,4.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
4,66217,220,0.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
anadf['CREB1'].sum()

8.0

In [16]:
trean_df = trelen_df

#retreive denominator for singlet global probability
tre_denoms = []
for t in trean_df['tre_len'].values:
    t_dnom = 0
    for l in raw_df['seq_len'].values:
        seq_denom = (l-t)
        t_dnom += seq_denom
    tre_denoms.append(t_dnom)
trean_df['denom'] = tre_denoms

#retreive global singlet counts (numerator)
tre_cnt = pivdf.sum().reset_index()
tre_cnt.columns = ['tre_name', 'tre_count']

#calc singlet global probability
singlet_df = pd.merge(trean_df, tre_cnt, on='tre_name', how='inner')
singlet_df['singlet_prob'] = singlet_df.tre_count/ singlet_df.denom

print(singlet_df.shape)
singlet_df.head()

(166, 6)


,tre,tre_name,tre_len,denom,tre_count,singlet_prob
0,1,TFAP2A,11,243263,3.0,0.000012
1,7,NR2F1,13,240759,6.0,0.000025
2,8,CREB1,8,247019,8.0,0.000032
3,10,E2F1,12,242011,936.0,0.003868
4,11,NFIL3,11,243263,2.0,0.000008


In [17]:
possible_dublets = list((itertools.combinations(pivdf.columns[1:], 2)))
print(possible_dublets)

[('ARNT::HIF1A', 'ATF4'), ('ARNT::HIF1A', 'ATF7'), ('ARNT::HIF1A', 'Arid3a'), ('ARNT::HIF1A', 'Atf1'), ('ARNT::HIF1A', 'Atf3'), ('ARNT::HIF1A', 'BATF3'), ('ARNT::HIF1A', 'BATF::JUN'), ('ARNT::HIF1A', 'Bcl6'), ('ARNT::HIF1A', 'CEBPA'), ('ARNT::HIF1A', 'CREB1'), ('ARNT::HIF1A', 'CTCF'), ('ARNT::HIF1A', 'CUX1'), ('ARNT::HIF1A', 'CUX2'), ('ARNT::HIF1A', 'Creb3l2'), ('ARNT::HIF1A', 'Creb5'), ('ARNT::HIF1A', 'Crem'), ('ARNT::HIF1A', 'DBP'), ('ARNT::HIF1A', 'E2F1'), ('ARNT::HIF1A', 'E2F6'), ('ARNT::HIF1A', 'EBF1'), ('ARNT::HIF1A', 'EGR1'), ('ARNT::HIF1A', 'EGR2'), ('ARNT::HIF1A', 'EGR3'), ('ARNT::HIF1A', 'EGR4'), ('ARNT::HIF1A', 'ELF5'), ('ARNT::HIF1A', 'ELK3'), ('ARNT::HIF1A', 'ELK4'), ('ARNT::HIF1A', 'EOMES'), ('ARNT::HIF1A', 'ESR1'), ('ARNT::HIF1A', 'ESR2'), ('ARNT::HIF1A', 'ESRRB'), ('ARNT::HIF1A', 'ETV4'), ('ARNT::HIF1A', 'ETV5'), ('ARNT::HIF1A', 'ETV6'), ('ARNT::HIF1A', 'Esrra'), ('ARNT::HIF1A', 'Esrrg'), ('ARNT::HIF1A', 'FEV'), ('ARNT::HIF1A', 'FOS'), ('ARNT::HIF1A', 'FOS::JUN'), ('ARN

In [18]:
#create dataframe with presence/absence of dublets for each sequence
dublet_df = pd.DataFrame()
for tup in possible_dublets:
    dublet_df[tup] = np.where((pivdf[tup[0]].values > 0) & (pivdf[tup[1]].values > 0), 1, 0)
dublet_df.head()

,"(ARNT::HIF1A, ATF4)","(ARNT::HIF1A, ATF7)","(ARNT::HIF1A, Arid3a)","(ARNT::HIF1A, Atf1)","(ARNT::HIF1A, Atf3)","(ARNT::HIF1A, BATF3)","(ARNT::HIF1A, BATF::JUN)","(ARNT::HIF1A, Bcl6)","(ARNT::HIF1A, CEBPA)","(ARNT::HIF1A, CREB1)",...,"(YY1, ZBTB7B)","(YY1, ZBTB7C)","(YY1, ZNF263)","(YY1, ZNF740)","(ZBTB7B, ZBTB7C)","(ZBTB7B, ZNF263)","(ZBTB7B, ZNF740)","(ZBTB7C, ZNF263)","(ZBTB7C, ZNF740)","(ZNF263, ZNF740)"
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
raw_df.shape

(1252, 4)

In [20]:
#store len of singlet TREs in a callable dict
len_dict = dict(zip(trelen_df['tre_name'], trelen_df['tre_len']))
singlet_dict = dict(zip(singlet_df['tre_name'], singlet_df['singlet_prob']))

In [21]:
#add weight and length variables, calc singlet probability FOR A SEQUENCE
dublet_df['weight'] = raw_df['count']
dublet_df['length'] = raw_df['seq_len']
for k, v in singlet_dict.items():
    dublet_df[k] = (1-((1-v)**(dublet_df['length'] - len_dict[k])))
dublet_df.head()

,"(ARNT::HIF1A, ATF4)","(ARNT::HIF1A, ATF7)","(ARNT::HIF1A, Arid3a)","(ARNT::HIF1A, Atf1)","(ARNT::HIF1A, Atf3)","(ARNT::HIF1A, BATF3)","(ARNT::HIF1A, BATF::JUN)","(ARNT::HIF1A, Bcl6)","(ARNT::HIF1A, CEBPA)","(ARNT::HIF1A, CREB1)",...,ATF7,BATF3,Creb5,NFE2,RXRB,RXRG,Rarb,Rarg,TFAP2A(var.3),HOXB3
0,0,0,0,0,0,0,0,0,0,0,...,0.964669,0.009028,0.965427,0.611863,0.000755,0.000755,0.001509,0.002263,0.442630,0.000756
1,0,0,0,0,0,0,0,0,0,0,...,0.990993,0.012695,0.991131,0.734731,0.001064,0.001064,0.002133,0.003197,0.560532,0.001059
2,0,0,0,0,0,0,0,0,0,0,...,0.983123,0.011012,0.983429,0.684038,0.000922,0.000922,0.001846,0.002768,0.509825,0.000920
3,0,0,0,0,0,0,0,0,0,0,...,0.955903,0.008432,0.956892,0.587151,0.000705,0.000705,0.001408,0.002112,0.420729,0.000707
4,0,0,0,0,0,0,0,0,0,0,...,0.977735,0.010268,0.978167,0.658696,0.000860,0.000860,0.001720,0.002579,0.485633,0.000858


In [22]:
list(np.where(dublet_df[('MGA', 'TBX4')] > 0))

[array([436, 957], dtype=int64)]

In [23]:
dublet_df[('MGA', 'TBX4')][436]

1

In [24]:
#query tuple headers with headers of their elements, subtract multiplied singlet probabilities from dublet binaries
tuple_columns = [col for col in dublet_df.columns if isinstance(col, tuple)]
probs_success_samples = pd.concat([dublet_df['weight'] * dublet_df[col[0]] * dublet_df[col[1]] for col in tuple_columns], axis=1)
#probs_success_samples = pd.concat([dublet_df[col[0]] * dublet_df[col[1]] for col in tuple_columns], axis=1)
probs_success_samples.columns = tuple_columns
#probs_success_samples = probs_success_samples.loc[:, (probs_success_samples != 0).all()]

var_probs_samples = pd.concat([dublet_df['weight'] * (dublet_df[col[0]] * dublet_df[col[1]]) * (1 - (dublet_df[col[0]] * dublet_df[col[1]])) for col in tuple_columns], axis=1)
var_probs_samples.columns = tuple_columns
#var_probs_samples = var_probs_samples.loc[:, (var_probs_samples != 0).all()]
    
observed_success_samples = pd.concat([dublet_df['weight'] * dublet_df[col] for col in tuple_columns], axis=1)
observed_success_samples.columns = tuple_columns
#observed_success_samples.apply(pd.to_numeric, errors='coerce')
#observed_success_samples_new = pd.DataFrame()
#for col in observed_success_samples.columns:
#    if observed_success_samples[col].sum() != 0:
#        observed_success_samples_new[col] = observed_success_samples[col]
#observed_success_samples_new = observed_success_samples.loc[:, (observed_success_samples != 0).all()] #for some reason this is bugged

In [25]:
probs_success_samples[('Creb5', 'STAT3')].sum()

869133.219022089

In [37]:
probs_success_samples[80:90].head()

,"(ARNT::HIF1A, ATF4)","(ARNT::HIF1A, ATF7)","(ARNT::HIF1A, Arid3a)","(ARNT::HIF1A, Atf1)","(ARNT::HIF1A, Atf3)","(ARNT::HIF1A, BATF3)","(ARNT::HIF1A, BATF::JUN)","(ARNT::HIF1A, Bcl6)","(ARNT::HIF1A, CEBPA)","(ARNT::HIF1A, CREB1)",...,"(YY1, ZBTB7B)","(YY1, ZBTB7C)","(YY1, ZNF263)","(YY1, ZNF740)","(ZBTB7B, ZBTB7C)","(ZBTB7B, ZNF263)","(ZBTB7B, ZNF740)","(ZBTB7C, ZNF263)","(ZBTB7C, ZNF740)","(ZNF263, ZNF740)"
80,0.513414,66.722634,0.051647,0.411806,0.770140,0.615404,42.031287,64.789487,0.103065,0.411806,...,0.033596,0.030553,0.573485,0.511811,0.041964,0.787668,0.702960,0.716326,0.639291,11.999415
81,1.287690,105.619304,0.127903,1.022406,1.908952,1.546123,83.002789,104.906538,0.257873,1.022406,...,0.087377,0.079482,1.453573,1.275922,0.109089,1.995031,1.751205,1.814765,1.592970,29.132514
82,0.705180,77.463992,0.070529,0.563083,1.052500,0.845945,53.271088,76.032435,0.141381,0.563083,...,0.046723,0.042494,0.792790,0.701754,0.058351,1.088630,0.963622,0.990105,0.876411,16.350787
83,0.486826,62.961675,0.048963,0.390422,0.730139,0.583550,39.766151,61.159009,0.097724,0.390422,...,0.031868,0.028981,0.543914,0.485284,0.039805,0.747049,0.666522,0.679388,0.606155,11.376138
84,0.279631,45.028308,0.028422,0.226059,0.423009,0.334679,25.139790,42.939144,0.056281,0.226059,...,0.018017,0.016383,0.307845,0.279158,0.022508,0.422925,0.383514,0.384586,0.348748,6.552959


In [27]:
var_probs_samples[('Creb5', 'STAT3')].sum()

95081.21998165485

In [36]:
observed_success_samples[80:90].head()

,"(ARNT::HIF1A, ATF4)","(ARNT::HIF1A, ATF7)","(ARNT::HIF1A, Arid3a)","(ARNT::HIF1A, Atf1)","(ARNT::HIF1A, Atf3)","(ARNT::HIF1A, BATF3)","(ARNT::HIF1A, BATF::JUN)","(ARNT::HIF1A, Bcl6)","(ARNT::HIF1A, CEBPA)","(ARNT::HIF1A, CREB1)",...,"(YY1, ZBTB7B)","(YY1, ZBTB7C)","(YY1, ZNF263)","(YY1, ZNF740)","(ZBTB7B, ZBTB7C)","(ZBTB7B, ZNF263)","(ZBTB7B, ZNF740)","(ZBTB7C, ZNF263)","(ZBTB7C, ZNF740)","(ZNF263, ZNF740)"
80,0,695,0,0,0,0,695,695,0,0,...,0,0,0,0,0,0,0,0,0,0
81,0,676,0,0,0,0,676,676,0,0,...,0,0,0,0,0,0,0,0,0,0
82,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
83,0,0,0,0,0,0,0,652,0,0,...,0,0,0,0,0,0,0,0,0,0
84,0,0,0,0,0,0,624,624,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
#print(list(norm_hypothesis_test(probs_success_samples, var_probs_samples, observed_success_samples)))

In [30]:
#given two datasets (expected vs observed), calculate normal CDF of expected and return p-value for observed
def norm_hypothesis_test(mean_probability_df, var_probability_df, success_df):
    u = mean_probability_df.sum()
    o = var_probability_df.sum()
    return scipy.stats.norm(u, o).cdf(success_df.sum())
    
#Calculate and sort p-values for a tuple of TREs
p_values = pd.DataFrame(norm_hypothesis_test(probs_success_samples, var_probs_samples, observed_success_samples))
p_values.columns = ['p_value']
p_values['tuples'] = observed_success_samples.columns
p_values.sort_values('p_value', ascending=True, inplace=True)

In [31]:
forchord = p_values.loc[p_values['p_value'] < 0.05]
forchord['nameA'] = [x[0] for x in forchord['tuples'].values]
forchord['nameB'] = [x[1] for x in forchord['tuples'].values]
forchord['invp'] = [int((0.05-x)*100) for x in forchord['p_value'].values]
duplicatelst = ['REL', 'CUX2', 'TFAP2B(var.3)', 'TFAP2C(var.3)']
for n in duplicatelst:
    forchord = forchord.loc[(forchord['nameA'] != n) & (forchord['nameB'] != n)]
    print(forchord.shape)
forchord.head()

(79, 5)
(78, 5)
(74, 5)
(59, 5)


C:\Users\Wolf\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\Wolf\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Users\Wolf\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: h

,p_value,tuples,nameA,nameB,invp
2488,0.000179,"(Creb5, STAT3)",Creb5,STAT3,4
2486,0.000182,"(Creb5, STAT1)",Creb5,STAT1,4
7885,0.000186,"(JDP2(var.2), STAT3)",JDP2(var.2),STAT3,4
7883,0.000190,"(JDP2(var.2), STAT1)",JDP2(var.2),STAT1,4
2489,0.000194,"(Creb5, Stat4)",Creb5,Stat4,4


In [32]:
from bokeh.palettes import inferno
chord_novalinf = Chord(forchord, source='nameA', target='nameB', palette= inferno(2))
output_file('chord_novalinf.html', mode="inline")
show(chord_novalinf)

In [81]:
from bokeh.charts import Histogram
invhist = Histogram(forchord, 'invp', palette= inferno(50))
show(invhist)

In [66]:
chord_wval = Chord(forchord, source='nameA', target='nameB', value='invp')
output_file('chord_wval.html', mode="inline")
show(chord_wval)

In [121]:
famforchord = forchord[['nameA', 'nameB']]
stat_lst = ['STAT_Family', 'STAT1', 'STAT3', 'Stat4', 'Stat5a::Stat5b']
jun_lst = ['cJun_Family', 'JUND(var.2)','JUN(var.2)', 'JUND', 'JUNB', 'JUN']
fos_lst = ['cFos_Family', 'FOS', 'FOSL1', 'FOSL2']
jd_lst = ['JDP Family', 'JDP2(var.2)','JDP2']
#tfap_lst = ['TFAP2A(var.3)', 'TFAP2B(var.3)', 'TFAP2C(var.3)']

#replace specific TRE with their family
conden_lst = [stat_lst, jun_lst, fos_lst, jd_lst]
for lst in conden_lst:
    famforchord.replace(regex=False, inplace=True, to_replace=lst[1:], value=lst[0])
famforchord = famforchord.reset_index()
p_tups = [tuple(x) for x in famforchord[['nameA', 'nameB']].values]
p_tups = dict(Counter(p_tups).items())
p = [(k[0], k[1], v) for k, v in p_tups.items()]
fam_df = pd.DataFrame(p, columns=['nameA', 'nameB', 'value'])


famforchord

C:\Users\Wolf\Anaconda3\lib\site-packages\ipykernel\__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,index,nameA,nameB
0,2488,Creb5,STAT_Family
1,2486,Creb5,STAT_Family
2,7885,JDP Family,STAT_Family
3,7883,JDP Family,STAT_Family
4,2489,Creb5,STAT_Family
5,7886,JDP Family,STAT_Family
6,460,ATF7,STAT_Family
7,458,ATF7,STAT_Family
8,461,ATF7,STAT_Family
9,2490,Creb5,STAT_Family


In [122]:
chord_fam = Chord(famforchord, source='nameA', target='nameB')
#output_file('chord_chordfam3.html', mode="inline")
show(chord_fam)

IndexError: pop from empty list

Probability of Drawing TRE = $\frac{\sum_{Seq}^{}Occurances of TRE in Sequence}{\sum_{Seq}^{}Length_{Sequence} - Length_{TRE}}$

Probability of a Sequence having a specific TRE = $1 - (1 - P_{TRE})^{Length_{Sequence} - Length_{TRE}}$